# Data Engineering Nanodegree - Capstone Project

### Project Summary
This project aims to structure an optimized analytical database of U.S. immigration events.<br>
The main objective is to create an ETL pipeline to build up a Data Warehouse working as a single source of data, facilitating the analysis through ad-hoc queries and helping Businesse Intelligence team to analyze treds and patterns of the immigration events.


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [228]:
# Import libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Row
from pyspark.sql.types import *

## Step 1: Scope the Project and Gather Data

### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>


### Datasets

#### [I-94 Immigration Data](https://www.trade.gov/i-94-arrivals-historical-data)
This dataset comes from the US National Tourism and Trade Office and contains information about immigrants in the U.S. <br>
A data dictionary is included in the workspace on the following path (dictionary).

##### Data Dictionary
| Column | Description |
| :--- | :--- |
| CICID | Unique ID |
| I94YR | 4-digit year |
| I94MON | Numeric month |
| I94CIT | 3-digit code of source city for immigration |
| I94RES | 3-digit code of source country for immigration |
| I94PORT | Port admitted through |
| ARRDATE | Arrival date in the USA |
| I94MODE | Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported) |
| I94ADDR | State of arrival |
| DEPDATE | Departure date from the USA |
| I94BIR | Age of Respondent in Years |
| I94VISA | Visa codes (1 = Business; 2 = Pleasure; 3 = Student) |
| COUNT | Used for summary statistics |
| DTADFILE | Character Date Field |
| VISAPOST | Department of State where Visa was issued |
| OCCUP | Occupation that will be performed in U.S. |
| ENTDEPA | Arrival Flag. Whether admitted or paroled into the US |
| ENTDEPD | Departure Flag. Whether Departed, lost I-94 or is deceased |
| ENTDEPU | Update Flag. Update of visa, Either apprehended, overstayed, adjusted to perm residence |
| MATFLAG | Match flag - Match of arrival and departure records|
| BIRYEAR | 4-digit year of birth |
| DTADDTO | Character date field to when admitted in the US (allowed to stay until) |
| GENDER | Gender |
| INSNUM | INS number |
| AIRLINE | Airline used to arrive in U.S. |
| ADMNUM | Admission number |
| FLTNO | Flight number of Airline used to arrive in U.S. |
| VISATYPE | Class of admission legally admitting the non-immigrant to temporarily stay in U.S. |

In [2]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_data = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
immigration_data.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


#### [Airport Code](https://datahub.io/core/airport-codes#data)
This dataset contains airport codes and corresponding cities.
##### Data Dictionary
|Column       |Description
|:-------------|:-----------
|ident         |Unique ID
|type          |Airport type
|name          |Airport name
|elevation_ft  |Airport altitude
|continent     |Continent
|iso_country   |Country Code
|iso_region    |Region Conde
|municipality  |City
|gps_code      |Airport GPS Code
|iata_code     |Airport IATA Code
|local_code    |Airport local code
|coordinates   |Airport coordinates

In [3]:
airport_data = pd.read_csv('raw_data/airport-codes_csv.csv')
airport_data.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


#### [U.S. City Demographic Data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)
This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000.<br> 
The data comes from the US Census Bureau's 2015 American Community Survey.
##### Data Dictionary
|Column                       |Description
|:-----------------------------|:-----------
|City |City Name
|State |US State of the City
|Median Age |The median population age
|Male Population |Male population total
|Female Population |Female population total
|Total Population |Total population
|Number of Veterans |Number of veterans living in the city
|Foreign-born |Number of residents who were not born in the city
|Average Household Size |Average size of houses in the city
|State Code |Code of the state
|Race |Race class
|Count |Number of individuals in each race"

In [182]:
demo = pd.read_csv('raw_data/us-cities-demographics.csv', delimiter=';')
demo.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


#### [World Temperature Data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)
The raw data comes from the [Berkeley Earth data page](https://berkeleyearth.org/data/).<br>
This dataset is a slice from the Global Temperatures Dataset and include Global Land Temperatures By City.

##### Data Dictionary
|Column                        |Description
|:-----------------------------|:-----------
|dt                            |Reference Date
|AverageTemperature            |Average temperature in Celsius
|AverageTemperatureUncertainty |95% confidence interval around average temperature
|City                          |City
|Country                       |Country
|Latitude                      |Latitude
|Longitude                     |Longitude

In [5]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temp = pd.read_csv(fname)
temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


## Step 2: Explore and Assess the Data

### Data Cleaning & Exploratory Analysis

- **Airport Data**

In [6]:
airport_data.shape

(55075, 12)

In [7]:
# Check data types
airport_data.dtypes

ident            object
type             object
name             object
elevation_ft    float64
continent        object
iso_country      object
iso_region       object
municipality     object
gps_code         object
iata_code        object
local_code       object
coordinates      object
dtype: object

In [8]:
# Check Null values per column
((airport_data.isnull().sum()/airport_data.shape[0])*100).sort_values(ascending=False)

iata_code       83.315479
continent       50.329551
local_code      47.914662
gps_code        25.501589
elevation_ft    12.720835
municipality    10.305946
iso_country      0.448479
coordinates      0.000000
iso_region       0.000000
name             0.000000
type             0.000000
ident            0.000000
dtype: float64

In [9]:
# Drop columns not used
columns = ['iata_code', 'continent', 'gps_code', 'elevation_ft', 'local_code']
airport_data = airport_data.drop(columns=columns)

In [10]:
# Check Null values per column
((airport_data.isnull().sum()/airport_data.shape[0])*100).sort_values(ascending=False)

municipality    10.305946
iso_country      0.448479
coordinates      0.000000
iso_region       0.000000
name             0.000000
type             0.000000
ident            0.000000
dtype: float64

In [11]:
airport_data.loc[(airport_data.iso_country.isnull())]

,ident,type,name,iso_country,iso_region,municipality,coordinates
21422,FYAA,small_airport,Ai-Ais Airport,NaN,NA-KU,Ai-Ais,"17.5966, -27.995"
21423,FYAB,small_airport,Aroab B Airport,NaN,NA-KA,Aroab,"19.633100509643555, -26.776100158691406"
21424,FYAK,small_airport,Aussenkehr Airport,NaN,NA-KA,Aussenkehr,"17.4645, -28.4587"
21425,FYAM,small_airport,Aminuis Airstrip,NaN,NA-OH,Aminuis,"19.351699829101562, -23.655799865722656"
21426,FYAR,medium_airport,Arandis Airport,NaN,NA-ER,Arandis,"14.979999542236328, -22.462200164794922"
21427,FYAS,small_airport,Aus Airport,NaN,NA-KA,Aus,"16.318899154663086, -26.693899154663086"
21428,FYAV,small_airport,Ariamsvley Airport,NaN,NA-KA,Ariamsvley,"19.833900451660156, -28.118900299072266"
21429,FYBC,small_airport,Bethanien Airport,NaN,NA-KA,Bethanien,"17.1816, -26.5448"
21430,FYBJ,small_airport,Bitterwasser Lodge & Flying Club Airfield,NaN,NA-HA,Bitterwasser,"17.9911003113, -23.875"
21431,FYEK,small_airport,Epukiro Airport,NaN,NA-OH,Epukiro,"19.10610008239746, -21.786699295043945"


In [12]:
mask = (airport_data.iso_country.isnull()) & (airport_data.iso_region.str[:2] == 'NA')

airport_data.loc[mask, 'iso_country'] = 'NA'

In [13]:
((airport_data.isnull().sum()/airport_data.shape[0])*100).sort_values(ascending=False)

municipality    10.305946
coordinates      0.000000
iso_region       0.000000
iso_country      0.000000
name             0.000000
type             0.000000
ident            0.000000
dtype: float64

In [14]:
airport_data.duplicated().sum()

0

In [15]:
airport_data.dropna(axis=0, inplace=True)

In [16]:
airport_data.shape

(49399, 7)

- **Demographic Data**

In [17]:
demo.shape

(2891, 12)

In [18]:
# Check data types
demo.dtypes

City                       object
State                      object
Median Age                float64
Male Population           float64
Female Population         float64
Total Population            int64
Number of Veterans        float64
Foreign-born              float64
Average Household Size    float64
State Code                 object
Race                       object
Count                       int64
dtype: object

In [19]:
# Check Null values per column
((demo.isnull().sum()/demo.shape[0])*100).sort_values(ascending=False)

Average Household Size    0.553442
Foreign-born              0.449671
Number of Veterans        0.449671
Female Population         0.103770
Male Population           0.103770
Count                     0.000000
Race                      0.000000
State Code                0.000000
Total Population          0.000000
Median Age                0.000000
State                     0.000000
City                      0.000000
dtype: float64

In [20]:
demo = demo.drop(columns='Number of Veterans')
demo.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,86253.0,2.73,NJ,White,76402


In [183]:
demo.loc[(demo.City=='Portland')]

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
530,Portland,Maine,40.3,31480.0,35392.0,66872,3666.0,9229.0,2.13,ME,White,59722
943,Portland,Oregon,36.7,313516.0,318671.0,632187,29940.0,86041.0,2.43,OR,American Indian and Alaska Native,15314
1962,Portland,Maine,40.3,31480.0,35392.0,66872,3666.0,9229.0,2.13,ME,Black or African-American,5750
2534,Portland,Oregon,36.7,313516.0,318671.0,632187,29940.0,86041.0,2.43,OR,White,524258
2535,Portland,Maine,40.3,31480.0,35392.0,66872,3666.0,9229.0,2.13,ME,Asian,3696
2724,Portland,Oregon,36.7,313516.0,318671.0,632187,29940.0,86041.0,2.43,OR,Asian,64386
2793,Portland,Oregon,36.7,313516.0,318671.0,632187,29940.0,86041.0,2.43,OR,Hispanic or Latino,61064
2794,Portland,Maine,40.3,31480.0,35392.0,66872,3666.0,9229.0,2.13,ME,Hispanic or Latino,2031
2795,Portland,Maine,40.3,31480.0,35392.0,66872,3666.0,9229.0,2.13,ME,American Indian and Alaska Native,662
2831,Portland,Oregon,36.7,313516.0,318671.0,632187,29940.0,86041.0,2.43,OR,Black or African-American,45914


In [22]:
demo.loc[(demo['Female Population'].isnull() | demo['Male Population'].isnull())]

,City,State,Median Age,Male Population,Female Population,Total Population,Foreign-born,Average Household Size,State Code,Race,Count
333,The Villages,Florida,70.5,NaN,NaN,72590,4034.0,NaN,FL,Hispanic or Latino,1066
449,The Villages,Florida,70.5,NaN,NaN,72590,4034.0,NaN,FL,Black or African-American,331
1437,The Villages,Florida,70.5,NaN,NaN,72590,4034.0,NaN,FL,White,72211


In [23]:
demo.loc[(demo['Foreign-born'].isnull() | demo['Average Household Size'].isnull())]

,City,State,Median Age,Male Population,Female Population,Total Population,Foreign-born,Average Household Size,State Code,Race,Count
111,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,PR,Hispanic or Latino,335559
155,Caguas,Puerto Rico,40.4,34743.0,42265.0,77008,NaN,NaN,PR,Hispanic or Latino,76349
258,Carolina,Puerto Rico,42.0,64758.0,77308.0,142066,NaN,NaN,PR,American Indian and Alaska Native,12143
333,The Villages,Florida,70.5,NaN,NaN,72590,4034.0,NaN,FL,Hispanic or Latino,1066
449,The Villages,Florida,70.5,NaN,NaN,72590,4034.0,NaN,FL,Black or African-American,331
637,Carolina,Puerto Rico,42.0,64758.0,77308.0,142066,NaN,NaN,PR,Hispanic or Latino,139967
1437,The Villages,Florida,70.5,NaN,NaN,72590,4034.0,NaN,FL,White,72211
1747,San Juan,Puerto Rico,41.4,155408.0,186829.0,342237,NaN,NaN,PR,American Indian and Alaska Native,4031
1748,Mayagüez,Puerto Rico,38.1,30799.0,35782.0,66581,NaN,NaN,PR,Asian,235
1995,Ponce,Puerto Rico,40.5,56968.0,64615.0,121583,NaN,NaN,PR,Hispanic or Latino,120705


In [ ]:
demo.dropna(axis=0, inplace=True)

In [24]:
demo.duplicated().sum()

0

In [25]:
demo.shape

(2891, 11)

In [26]:
demo.columns = [col.lower().replace(' ', '_').replace('-', '_') for col in demo.columns]

In [27]:
demo.head()

,city,state,median_age,male_population,female_population,total_population,foreign_born,average_household_size,state_code,race,count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,86253.0,2.73,NJ,White,76402


In [28]:
demo['city'] = demo['city'].str.upper()
demo['state'] = demo['state'].str.upper()
demo.head(5)

,city,state,median_age,male_population,female_population,total_population,foreign_born,average_household_size,state_code,race,count
0,SILVER SPRING,MARYLAND,33.8,40601.0,41862.0,82463,30908.0,2.60,MD,Hispanic or Latino,25924
1,QUINCY,MASSACHUSETTS,41.0,44129.0,49500.0,93629,32935.0,2.39,MA,White,58723
2,HOOVER,ALABAMA,38.5,38040.0,46799.0,84839,8229.0,2.58,AL,Asian,4759
3,RANCHO CUCAMONGA,CALIFORNIA,34.5,88127.0,87105.0,175232,33878.0,3.18,CA,Black or African-American,24437
4,NEWARK,NEW JERSEY,34.6,138040.0,143873.0,281913,86253.0,2.73,NJ,White,76402


- **Temperature Data**

In [29]:
temp.shape

(8599212, 7)

In [30]:
temp.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

In [31]:
((temp.isnull().sum()/temp.shape[0])*100).sort_values(ascending=False)

AverageTemperatureUncertainty    4.234458
AverageTemperature               4.234458
Longitude                        0.000000
Latitude                         0.000000
Country                          0.000000
City                             0.000000
dt                               0.000000
dtype: float64

In [32]:
print(f'Min. DT: {temp.dt.min()}')
print(f'Max. DT: {temp.dt.max()}')

Min. DT: 1743-11-01
Max. DT: 2013-09-01


In [33]:
# Create a tempora slice of 5 years
lower_dt = '2008-01-01'
temp = temp.loc[temp.dt >= lower_dt]

In [34]:
((temp.isnull().sum()/temp.shape[0])*100).sort_values(ascending=False)

AverageTemperatureUncertainty    1.2676
AverageTemperature               1.2676
Longitude                        0.0000
Latitude                         0.0000
Country                          0.0000
City                             0.0000
dt                               0.0000
dtype: float64

In [35]:
temp.loc[temp.AverageTemperature.isnull()]

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
3238,2013-09-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
6477,2013-09-01,NaN,NaN,Çorlu,Turkey,40.99N,27.69E
9606,2013-09-01,NaN,NaN,Çorum,Turkey,40.99N,34.08E
11924,2013-09-01,NaN,NaN,Öskemen,Kazakhstan,50.63N,82.39E
14242,2013-09-01,NaN,NaN,Ürümqi,China,44.20N,87.20E
17481,2013-09-01,NaN,NaN,A Coruña,Spain,42.59N,8.73W
20720,2013-09-01,NaN,NaN,Aachen,Germany,50.63N,6.34E
23959,2013-09-01,NaN,NaN,Aalborg,Denmark,57.05N,10.33E
25852,2013-09-01,NaN,NaN,Aba,Nigeria,5.63N,8.07E
28170,2013-09-01,NaN,NaN,Abadan,Iran,29.74N,48.00E


In [36]:
temp = temp.dropna(axis=0)

In [37]:
temp.shape

(239120, 7)

In [38]:
temp.columns = [''.join(['_' + c.lower() if c.isupper() else c for c in col]).lstrip('_') for col in temp.columns]

In [39]:
temp['city'] = temp['city'].str.upper()
temp.head(5)

,dt,average_temperature,average_temperature_uncertainty,city,country,latitude,longitude
3170,2008-01-01,3.953,0.383,ÅRHUS,Denmark,57.05N,10.33E
3171,2008-02-01,4.925,0.363,ÅRHUS,Denmark,57.05N,10.33E
3172,2008-03-01,3.654,0.172,ÅRHUS,Denmark,57.05N,10.33E
3173,2008-04-01,7.758,0.195,ÅRHUS,Denmark,57.05N,10.33E
3174,2008-05-01,13.243,0.220,ÅRHUS,Denmark,57.05N,10.33E


- **Immigration Events**

In [40]:
immigration_data.shape

(3096313, 28)

In [55]:
# Select relevant columns for this project purpose
immigration = immigration_data[[
    'cicid', 'i94yr', 'i94mon', 'i94res', 'i94port',
    'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
    'gender'
]]

In [54]:
immigration.head(5)

,cicid,i94yr,i94mon,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,dtaddto,gender,airline,visatype
0,6.0,2016.0,4.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,10282016,NaN,NaN,B2
1,7.0,2016.0,4.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,D/S,M,NaN,F1
2,15.0,2016.0,4.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,09302016,M,OS,B2
3,16.0,2016.0,4.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,09302016,NaN,AA,B2
4,17.0,2016.0,4.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,09302016,NaN,AA,B2


In [42]:
immigration.dtypes

cicid       float64
i94yr       float64
i94mon      float64
i94res      float64
i94port      object
arrdate     float64
i94mode     float64
i94addr      object
depdate     float64
i94bir      float64
i94visa     float64
dtaddto      object
gender       object
airline      object
visatype     object
dtype: object

In [43]:
print(f'Max yr: {immigration.i94yr.max()}')
print(f'Min yr: {immigration.i94yr.min()}')
print(f'Max mon: {immigration.i94mon.max()}')
print(f'Min mon: {immigration.i94mon.min()}')

Max yr: 2016.0
Min yr: 2016.0
Max mon: 4.0
Min mon: 4.0


In [44]:
((immigration.isnull().sum()/immigration.shape[0])*100).sort_values(ascending=False)

gender      13.379429
i94addr      4.921079
depdate      4.600859
airline      2.700857
i94bir       0.025902
dtaddto      0.015405
i94mode      0.007719
visatype     0.000000
i94visa      0.000000
arrdate      0.000000
i94port      0.000000
i94res       0.000000
i94mon       0.000000
i94yr        0.000000
cicid        0.000000
dtype: float64

In [45]:
immigration.head()

,cicid,i94yr,i94mon,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,dtaddto,gender,airline,visatype
0,6.0,2016.0,4.0,692.0,XXX,20573.0,NaN,NaN,NaN,37.0,2.0,10282016,NaN,NaN,B2
1,7.0,2016.0,4.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,D/S,M,NaN,F1
2,15.0,2016.0,4.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,09302016,M,OS,B2
3,16.0,2016.0,4.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,09302016,NaN,AA,B2
4,17.0,2016.0,4.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,09302016,NaN,AA,B2


In [46]:
immigration.shape

(3096313, 15)

#### Parse description file to get auxiliary information

In [47]:
with open("raw_data/I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()

In [48]:
country_code = {}
for countries in contents[10:298]:
    pair = countries.split('=')
    code, country = pair[0].strip(), pair[1].strip().strip("'")
    country_code[code] = country

In [49]:
country_data = pd.DataFrame(list(country_code.items()), columns=['code', 'country'])
country_data.head(5)

,code,country
0,236,AFGHANISTAN
1,101,ALBANIA
2,316,ALGERIA
3,102,ANDORRA
4,324,ANGOLA


In [65]:
city_code = {}
for cities in contents[303:962]:
    pair = cities.split('=')
    code, city = pair[0].strip("\t").strip().strip("'"), pair[1].strip('\t').strip().strip("''")
    city_code[code] = city

In [51]:
city_data = pd.DataFrame(list(city_code.items()), columns=['code', 'city'])
city_data.head(5)

,code,city
0,ANC,"ANCHORAGE, AK"
1,BAR,"BAKER AAF - BAKER ISLAND, AK"
2,DAC,"DALTONS CACHE, AK"
3,PIZ,"DEW STATION PT LAY DEW, AK"
4,DTH,"DUTCH HARBOR, AK"


In [52]:
state_code = {}
for states in contents[982:1036]:
    pair = states.split('=')
    code, state = pair[0].strip('\t').strip("'"), pair[1].strip().strip("'")
    state_code[code] = state

In [53]:
state_data = pd.DataFrame(list(state_code.items()), columns=['code', 'state'])
state_data.head(5)

,code,state
0,AK,ALASKA
1,AZ,ARIZONA
2,AR,ARKANSAS
3,CA,CALIFORNIA
4,CO,COLORADO


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

- Multidimensional Model
![title](img/dend-udacity.png)

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

- Select relevant features to construct immigration_fact based on I94 raw data
    - clean up incorrect or missing data
    - convert arrdate and depdate into standard/readable DATE format
- Create Dimension tables
    - dim_demo (Based on demographics data)
    - dim_mode (Based on auxiliary data provided)
    - dim_visa (Based on visa data)
    - dim_country (Based on auxiliary date provided)
    - dim_port (Based on auxiliary data provided)
- Write Dimension tables into Parquet format
- Write Fact table into Parquet format

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

### Read Data With Spark

In [5]:
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [229]:
df_spark=spark.read.parquet("raw_data/sas_data")

In [230]:
immigration_fact = df_spark.select(
    'cicid', 'i94yr', 'i94mon', 'i94res', 'i94port',
    'arrdate', 'depdate', 'i94mode', 'i94bir', 'i94visa', 'gender'
)

In [231]:
immigration_fact = immigration_fact.withColumn('arrdate', F.expr("date_add(to_date('1960-01-01'), int(arrdate))"))

In [232]:
immigration_fact = immigration_fact.withColumn('depdate', F.expr("date_add(to_date('1960-01-01'), int(depdate))"))

In [233]:
immigration_fact.show(5)

+---------+------+------+------+-------+----------+----------+-------+------+-------+------+
|    cicid| i94yr|i94mon|i94res|i94port|   arrdate|   depdate|i94mode|i94bir|i94visa|gender|
+---------+------+------+------+-------+----------+----------+-------+------+-------+------+
|5748517.0|2016.0|   4.0| 438.0|    LOS|2016-04-30|2016-05-08|    1.0|  40.0|    1.0|     F|
|5748518.0|2016.0|   4.0| 438.0|    LOS|2016-04-30|2016-05-17|    1.0|  32.0|    1.0|     F|
|5748519.0|2016.0|   4.0| 438.0|    LOS|2016-04-30|2016-05-08|    1.0|  29.0|    1.0|     M|
|5748520.0|2016.0|   4.0| 438.0|    LOS|2016-04-30|2016-05-14|    1.0|  29.0|    1.0|     F|
|5748521.0|2016.0|   4.0| 438.0|    LOS|2016-04-30|2016-05-14|    1.0|  28.0|    1.0|     M|
+---------+------+------+------+-------+----------+----------+-------+------+-------+------+
only showing top 5 rows



In [234]:
immigration_fact = immigration_fact.withColumn("cicid", F.col("cicid").cast(IntegerType())) \
                                        .withColumn("i94yr", F.col("i94yr").cast(IntegerType())) \
                                        .withColumn("i94mon", F.col("i94mon").cast(IntegerType())) \
                                        .withColumn("i94res", F.col("i94res").cast(IntegerType())) \
                                        .withColumn("i94mode", F.col("i94mode").cast(IntegerType())) \
                                        .withColumn("i94bir", F.col("i94bir").cast(IntegerType())) \
                                        .withColumn("i94visa", F.col("i94visa").cast(IntegerType()))

In [236]:
#write to parquet
immigration_fact.write.mode('overwrite').partitionBy('i94yr', 'i94mon').parquet("immigration_fact")

In [237]:
df_spark = spark.read.options(delimiter=';').csv('raw_data/us-cities-demographics.csv', header=True)

In [238]:
columns = df_spark.columns
new_columns = [col.lower().replace(' ', '_').replace('-', '_') for col in columns]
dim_demo = df_spark.toDF(*new_columns)

In [239]:
dim_demo = dim_demo.select(
    'city', 'state_code', 'median_age', 'male_population', 'female_population',
    'total_population', 'foreign_born', 'average_household_size'
)

In [240]:
dim_demo = dim_demo.withColumn('city', F.upper(F.col('city')))

In [241]:
dim_demo = dim_demo.withColumn('city_state', F.concat_ws('-',dim_demo.city,dim_demo.state_code))

In [242]:
dim_demo = dim_demo.select(
    'city_state', 'city', 'state_code', 'median_age', 
    'male_population', 'female_population',
    'total_population', 'foreign_born', 'average_household_size'
)

In [266]:
dim_demo = dim_demo.dropDuplicates()

In [267]:
dim_demo.show(5)

+----------------+-------------+----------+----------+---------------+-----------------+----------------+------------+----------------------+
|      city_state|         city|state_code|median_age|male_population|female_population|total_population|foreign_born|average_household_size|
+----------------+-------------+----------+----------+---------------+-----------------+----------------+------------+----------------------+
|        ALLEN-PA|        ALLEN|        PA|      33.5|          60626|            59581|          120207|       19652|                  2.67|
|     SYRACUSE-NY|     SYRACUSE|        NY|      30.3|          69462|            74690|          144152|       17733|                  2.39|
|        OGDEN-UT|        OGDEN|        UT|      31.3|          44323|            41127|           85450|       12791|                  2.68|
|BROOKLYN PARK-MN|BROOKLYN PARK|        MN|      35.1|          37845|            41305|           79150|       17490|                  2.85|
|  EAS

In [268]:
dim_demo.write.mode("overwrite").parquet("dim_demo")

In [245]:
df_spark = spark.read.options(delimiter=',').csv('raw_data/airport-codes_csv.csv', header=True)

In [246]:
dim_port = df_spark.filter(F.col('iso_country')=='US')

In [247]:
dim_port = dim_port.select('ident', 'iso_region', 'municipality')

In [248]:
dim_port = dim_port.withColumn('state', F.split(dim_port.iso_region, '-')[1])

In [249]:
dim_port = dim_port.drop('iso_region')

In [250]:
columns = ['port_code', 'city', 'state']
dim_port = dim_port.toDF(*columns)

In [251]:
dim_port = dim_port.withColumn('city', F.upper(dim_port.city))

In [252]:
dim_port = dim_port.withColumn('city_state', F.concat_ws('-', dim_port.city, dim_port.state))

In [253]:
dim_port.show(2)

+---------+--------+-----+-----------+
|port_code|    city|state| city_state|
+---------+--------+-----+-----------+
|      00A|BENSALEM|   PA|BENSALEM-PA|
|     00AA|   LEOTI|   KS|   LEOTI-KS|
+---------+--------+-----+-----------+
only showing top 2 rows



In [254]:
dim_port.write.mode("overwrite").parquet("dim_port")

In [255]:
visa_data = [
    {'visa_code': 1, 'visa_type': 'Business'},
    {'visa_code': 2, 'visa_type': 'Pleasure'},
    {'visa_code': 3, 'visa_type': 'Student'}
]

dim_visa = spark.createDataFrame(Row(**x) for x in visa_data)

In [256]:
dim_visa.show()

+---------+---------+
|visa_code|visa_type|
+---------+---------+
|        1| Business|
|        2| Pleasure|
|        3|  Student|
+---------+---------+



In [257]:
dim_visa.write.mode("overwrite").parquet('dim_visa')

In [258]:
with open("raw_data/I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()
    
countries = {}
for countrie in contents[10:298]:
    pair = countrie.split('=')
    country_code, country = pair[0].strip(), pair[1].strip().strip("'")
    countries[country_code] = country

dim_country = spark.createDataFrame(countries.items(), ['country_code', 'country'])

In [259]:
dim_country.show(5)

+------------+-----------+
|country_code|    country|
+------------+-----------+
|         236|AFGHANISTAN|
|         101|    ALBANIA|
|         316|    ALGERIA|
|         102|    ANDORRA|
|         324|     ANGOLA|
+------------+-----------+
only showing top 5 rows



In [260]:
dim_country.write.mode("overwrite").parquet('dim_country')

In [261]:
modes = {}
for mode in contents[973:976]:
    pair = mode.split('=')
    mode_id, mode_type = pair[0].strip(), pair[1].strip().strip("'").strip(";").strip("' ")
    modes[mode_id] = mode_type

dim_mode = spark.createDataFrame(modes.items(), ['mode_id', 'mode_type'])

In [262]:
dim_mode.show()

+-------+------------+
|mode_id|   mode_type|
+-------+------------+
|      2|         Sea|
|      3|        Land|
|      9|Not reported|
+-------+------------+



In [263]:
dim_mode.write.mode("overwrite").parquet('dim_mode')

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [269]:
list_dir = ['dim_country', 'dim_demo', 'dim_port', 'dim_mode', 'dim_visa', 'immigration_fact']
for file_dir in list_dir:
    #if file_dir.is_dir():
    path = str(file_dir)
    df = spark.read.parquet(path)
    record_num = df.count()
    if record_num <= 0:
        raise ValueError("This table is empty!")
    else:
        print("Table - " + path.split('/')[-1] + f" - Records: {record_num}")

Table - dim_country - Records: 288
Table - dim_demo - Records: 596
Table - dim_port - Records: 22757
Table - dim_mode - Records: 3
Table - dim_visa - Records: 3
Table - immigration_fact - Records: 3096313


In [270]:
list_dir = ['dim_country', 'dim_demo', 'dim_port', 'dim_mode', 'dim_visa', 'immigration_fact']

for file_dir in list_dir:
    path = str(file_dir)
    df = spark.read.parquet(path)
    unique = df.select(df.columns[0]).distinct().count()
    if df.count() > unique:
        raise ValueError(f"Table - {path} - Contains Duplicated Unique Key")
    else:
        print(f"Table - {path} - do not contains duplicated Unique Key")

Table - dim_country - do not contains duplicated Unique Key
Table - dim_demo - do not contains duplicated Unique Key
Table - dim_port - do not contains duplicated Unique Key
Table - dim_mode - do not contains duplicated Unique Key
Table - dim_visa - do not contains duplicated Unique Key
Table - immigration_fact - do not contains duplicated Unique Key


#### 4.3 Data dictionary 
Provided on README.md file

#### Step 5: Complete Project Write Up
1. Clearly state the rationale for the choice of tools and technologies for the project.
2. Propose how often the data should be updated and why.
3. Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

#### 5.1

The main purpose of this project is to process significant amount of immigration data provided aiming to construct an analytical environment optimized for Ad/Hoc queries.

Therefore Apache Spark is one of the best tools to deal with data processing once it distributes the workload over different nodes.<br>
The main ETL pipeline was constructed using Apache Spark through PySpark library which constructs analytical layer and enables data analysts to get insights of immigration in the U.S.

#### 5.2

It depends on how frequently it will be analyzed (business rule). But the data can be updated daily once I94 data is partitioned by year and month.

#### 5.3
- If the data was increased 100x it is advisable to increase the processing power using clusters with more nodes or cloud services e.g. AWS EMR which is a distributed data cluster for processing large amount of data.
- Schedule the ETL pipeline to regulary update data. It could be done using Apache Airflow to orchestrate all the process, including data qality checks.
- Data can be copied into Amazon Redshift, thus using RDS as the main analytical environment to store all the data and controll access thorugh IAM.